<a href="https://colab.research.google.com/github/snipaid-nlg/models/blob/main/text-summarization/news-summarization-with-mt5-small-sum-de-en-v2-on-gnad10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# News Summarization with mt5-small-sum-de-en-v2
Evaluating [mt5-small-sum-de-en-v2](https://huggingface.co/T-Systems-onsite/mt5-small-sum-de-en-v2) on the task of summarizing news in german language.

## Installs and Imports

In [ ]:
!pip install -q transformers datasets accelerate sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset, Dataset
from transformers.pipelines.pt_utils import KeyDataset
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM, pipeline
import pandas as pd
from tqdm.auto import tqdm

## Loading and transforming 10kGNAD
We load the [Ten Thousand German News Articles Dataset](https://tblock.github.io/10kGNAD/) from Huggingface. The first sentence of a text (up to the first period) is a headline, so we split the text and use the second part. A Prefix `summarize:` is added because the model was trained on the task of summarization.

In [ ]:
gnad10 = load_dataset("gnad10")
df = pd.DataFrame(gnad10["train"])
df[['headline', 'fulltext']] = df["text"].str.split('.', n=1, expand=True)
df.drop(['label','text'], axis=1, inplace=True)
df['fulltext'] = "summarize: " + df['fulltext']
df.head()

Generating train split:   0%|          | 0/9245 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1028 [00:00<?, ? examples/s]

Dataset gnad10 downloaded and prepared to /root/.cache/huggingface/datasets/gnad10/default/1.1.0/3a8445be65795ad88270af4d797034c3d99f70f8352ca658c586faf1cf960881. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

,headline,fulltext
0,21-Jähriger fällt wohl bis Saisonende aus,summarize: Wien – Rapid muss wohl bis Saisone...
1,"Erfundene Bilder zu Filmen, die als verloren g...",summarize: Wien – Die Filmgeschichte ist ein ...
2,Der frischgekürte CEO Sundar Pichai setzt auf ...,summarize: Die Atmosphäre im Silicon Valley i...
3,"Putin: ""Einigung, dass wir Menge auf Niveau vo...",summarize: Moskau – Die russischen Ölproduzen...
4,Estland sieht den künftigen österreichischen P...,summarize: Wien/Tallinn/Pskow – Die Eltern de...


Converting the pandas dataframe to a Huggingface dataset object

In [ ]:
dataset = Dataset.from_pandas(df)

## Loading the model

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("T-Systems-onsite/mt5-small-sum-de-en-v2", use_cache=True, device_map='auto')
tokenizer = T5Tokenizer.from_pretrained("T-Systems-onsite/mt5-small-sum-de-en-v2")

In [ ]:
template = {
    "num_beams": 1,
    "max_new_tokens": None,
    "max_length" : 80,
    "length_penalty": 0.6,
    "min_new_tokens": 5,
    "no_repeat_ngram_size": 0,
    "repetition_penalty": 1.0,
    "diversity_penalty": 0.0,
    "num_beam_groups": 1,
    "do_sample": False,
    "top_k": 50,
    "top_p": 1.0,
    "temperature": 1.0,
    "early_stopping": True,
    "pad_token_id": 0,
    "eos_token_id": 1,
    "num_return_sequences": 1
}

In [ ]:
pipe = pipeline("text2text-generation",
                device=0,
                model=model, 
                tokenizer=tokenizer, 
                **template
                )

In [ ]:
for out in tqdm(pipe(KeyDataset(dataset, "fulltext"))):
    print(out)

  0%|          | 0/9245 [00:00<?, ?it/s]

[{'generated_text': 'Der frühere österreichische Nationalspieler Thomas Murg fällt beim 0:4 gegen Mödling in die Krise. Der 21-Jährige erlitt einen Teilriss im Knie.'}]
[{'generated_text': 'The Forbidden Room ist ein Two-Reeler von 1913/14, der der arbeitswütige US-Regisseur Allan Dwan gedreht hat.'}]
[{'generated_text': 'Die Zeiten des harten Management-Stils sind bei Google vorbei. Das belegen zahlreiche ehemalige Mitarbeiter und Geschäftspartner.'}]
[{'generated_text': 'Die russischen Ölproduzenten wollen nach den Worten von Präsident Putin ihre Förderung in diesem Jahr einfrieren. Im Großen und Ganzen wurde eine Einigung erzielt, dass wir die Ölproduktion auf dem Niveau von Jänner halten werden.'}]
[{'generated_text': 'Die Eltern des künftigen Bundespräsidenten waren aus dem von Sowjets besetzten Estland geflohen, wo 1944 in Wien Sascha Van der Bellen zur Welt kam.'}]
[{'generated_text': 'Die Trodat Trotec Holding GmbH mit Sitz in Wels übernimmt zwei US-Unternehmen, im Laserbereich

KeyboardInterrupt: ignored